In [1]:
library(ArchR)
library(rtracklayer)
addArchRThreads(threads = 16)


                                                   / |
                                                 /    \
            .                                  /      |.
            \\\                              /        |.
              \\\                          /           `|.
                \\\                      /              |.
                  \                    /                |\
                  \\#####\           /                  ||
                ==###########>      /                   ||
                 \\##==......\    /                     ||
            ______ =       =|__ /__                     ||      \\\
        ,--' ,----`-,__ ___/'  --,-`-===================##========>
       \               '        ##_______ _____ ,--,__,=##,__   ///
        ,    __==    ___,-,__,--'#'  ==='      `-'    | ##,-/
        -,____,---'       \\####\\________________,--\\_##,/
           ___      .______        ______  __    __  .______      
          /   \     |   _ 

In [2]:
TSVtoBinnedMatrix_CombineSamples <- function(FilePath,AllFiles,Binned_bed,GenomeVersion){
    print(AllFiles)
    addArchRGenome(GenomeVersion)
    FileName = lapply(AllFiles,FUN=function(x) {str_split(x,pattern = '[.]')[[1]][1]})
    names(AllFiles) <- FileName
    InputFiles = paste(FilePath,AllFiles,sep='')

    # make arrow file: hdf5 used to store bin/gene/peak/...more of scATAC modularly
    ArrowFiles <- createArrowFiles(
        inputFiles = InputFiles,
        sampleNames = names(AllFiles),
        minTSS = 4, #Dont set this too high because you can always increase later
        minFrags = 1000, 
        addTileMat = TRUE,
        addGeneScoreMat = TRUE)

    # something about infering more than 1 cell per chamber, resulting in two different cells getting the same barcode
    doubScores <- addDoubletScores(
        input = ArrowFiles,
        k = 10, #Refers to how many cells near a "pseudo-doublet" to count.
        knnMethod = "UMAP", #Refers to the embedding to use for nearest neighbor search.
        LSIMethod = 1
    )

    # this is the directory to which the outputs of the script will be spitted out
    OD = "Allsamples"
    
    projtemp <- ArchRProject(
        ArrowFiles = ArrowFiles, 
        outputDirectory = OD,
        copyArrows = FALSE
    ) # create a archR project
    projtemp <- filterDoublets(ArchRProj = projtemp) # filter doublets based on doubScores
    projtemp <- addIterativeLSI(ArchRProj = projtemp, useMatrix = "TileMatrix", name = "IterativeLSI") # 'Iter-LSI = dimension reduction'
    projtemp <- addHarmony(ArchRProj = projtemp,
                           reducedDims = "IterativeLSI",
                           name = "Harmony",
                           groupBy = "Sample"
                          ) # 'Harmony = batch effect correction function', useful to do if more than 1 sample is being compared
    projtemp <- addClusters(input = projtemp, reducedDims = "IterativeLSI") # call clusters using Seurats clustering method
    projtemp <- addUMAP(ArchRProj = projtemp, reducedDims = "IterativeLSI") # UMAP using uwot package
    projtemp <- addImputeWeights(projtemp)
    p1 <- plotEmbedding(ArchRProj = projtemp, colorBy = "cellColData", name = "Sample", embedding = "UMAP")
    p2 <- plotEmbedding(ArchRProj = projtemp, colorBy = "cellColData", name = "Clusters", embedding = "UMAP")
    plotPDF(p1,p2, name = "Plot-UMAP-Sample-Clusters.pdf", ArchRProj = projtemp, addDOC = FALSE, width = 5, height = 5)

    projtemp <- addPeakSet(
        ArchRProj = projtemp,
        peakSet = Binned_bed,
        genomeAnnotation = getGenomeAnnotation(projtemp),
        force = FALSE
    ) # PeakSet = 10kb-bin
    
    projtemp <- addPeakMatrix(
        ArchRProj = projtemp, 
        ceiling = 1000
    ) #

    M1 <- getMatrixFromProject(
        ArchRProj = projtemp,
        useMatrix = "PeakMatrix",
    )

    M2 <- assays(M1)$PeakMatrix
    names = colnames(M2);
    M3 <- as.data.frame(summary(M2));
    nfilematrix = paste(OD,GenomeVersion,"binned_matrix.txt", sep = "_");
    nfilecells = paste(OD,GenomeVersion,"cells.txt", sep = "_");
    nfileclusters = paste(OD,GenomeVersion,"clusters.txt", sep = "_");

    write.table(M3, file = nfilematrix, col.names=FALSE, row.names=FALSE, quote = FALSE);
    write.table(names, file = nfilecells, col.names=FALSE, row.names=FALSE, quote = FALSE);
    write.table(projtemp$Clusters, file = nfileclusters, col.names=FALSE, row.names=FALSE, quote = FALSE); 

    saveArchRProject(ArchRProj = projtemp, outputDirectory = OD, load = FALSE)    
}

In [3]:
# reading 'fragment.tsv.gz' file: chr,start,end,barcode,readcount
filepath <- "/mnt/g/genomics/raw_tsv/CA1/"
allfiles = list.files(path = filepath, pattern = '.tsv.gz$', ignore.case = FALSE, include.dirs = FALSE, no.. = FALSE)

# reading bed file of 10kb binned mouse genome
mm10kb_bed <- import("/mnt/g/genomics/scATACcodes/MatlabCodes/MatlabCode/mm10_10k.txt",format="bed")


In [4]:
TSVtoBinnedMatrix_CombineSamples(FilePath=filepath,AllFiles=allfiles,Binned_bed=mm10kb_bed,GenomeVersion='mm10')

[1] "Glut_CA1GL1_sorted2.tsv.gz" "Glut_CA1GL2_sorted2.tsv.gz"
[3] "Glut_CA1GL3_sorted2.tsv.gz"


Setting default genome to Mm10.

Using GeneAnnotation set by addArchRGenome(Mm10)!

Using GeneAnnotation set by addArchRGenome(Mm10)!

ArchR logging to : ArchRLogs/ArchR-createArrows-84be42737f35-Date-2023-10-16_Time-10-13-04.810305.log
If there is an issue, please report to github with logFile!

Cleaning Temporary Files

2023-10-16 10:13:07.460171 : Batch Execution w/ safelapply!, 0 mins elapsed.

ArchR logging successful to : ArchRLogs/ArchR-createArrows-84be42737f35-Date-2023-10-16_Time-10-13-04.810305.log

ArchR logging to : ArchRLogs/ArchR-addDoubletScores-84be7c1c7136-Date-2023-10-16_Time-10-42-27.009828.log
If there is an issue, please report to github with logFile!

2023-10-16 10:42:28.611614 : Batch Execution w/ safelapply!, 0 mins elapsed.

2023-10-16 10:42:28.68679 : Glut_CA1GL1_sorted2 (1 of 3) :  Computing Doublet Statistics, 0.001 mins elapsed.

Glut_CA1GL1_sorted2 (1 of 3) : UMAP Projection R^2 = 0.98446

Glut_CA1GL1_sorted2 (1 of 3) : UMAP Projection R^2 = 0.98446

2023

Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 17190
Number of edges: 542293

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.7397
Number of communities: 8
Elapsed time: 1 seconds


2 singletons identified. 6 final clusters.

2023-10-16 10:52:27.475216 : Testing Biased Clusters, 0.237 mins elapsed.

2023-10-16 10:52:27.52822 : Testing Outlier Clusters, 0.238 mins elapsed.

2023-10-16 10:52:27.55981 : Assigning Cluster Names to 6 Clusters, 0.239 mins elapsed.

2023-10-16 10:52:27.679759 : Finished addClusters, 0.241 mins elapsed.

10:52:27 UMAP embedding parameters a = 0.7669 b = 1.223

10:52:27 Read 17190 rows and found 30 numeric columns

10:52:27 Using Annoy for neighbor search, n_neighbors = 40

10:52:27 Building Annoy index with metric = cosine, n_trees = 50

0%   10   20   30   40   50   60   70   80   90   100%

[----|----|----|----|----|----|----|----|----|----|

*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
|

10:52:28 Writing NN index file to temp file /tmp/RtmpenYnWt/file84be4536e9a4

10:52:28 Searching Annoy index using 24 threads, search_k = 4000

10:52:29 Annoy recall = 100%

10:52:30 Commencing sm